<a href="https://colab.research.google.com/github/czhurdlespeed/TFDevCert/blob/main/TFCert_Practice/08_introduction_to_nlp_in_tensorflow.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Introduction to NLP Fundamentals in TensorFlow

## Check for GPU

In [1]:
!nvidia-smi -L

GPU 0: Tesla T4 (UUID: GPU-839e28c2-84c3-f401-856a-b4c4ca8090ef)


## Import Helper Functions

In [2]:
!wget https://github.com/mrdbourke/tensorflow-deep-learning/raw/main/extras/helper_functions.py
from helper_functions import unzip_data, create_tensorboard_callback, plot_loss_curves, compare_historys

--2024-01-29 14:01:19--  https://github.com/mrdbourke/tensorflow-deep-learning/raw/main/extras/helper_functions.py
Resolving github.com (github.com)... 140.82.114.3
Connecting to github.com (github.com)|140.82.114.3|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/mrdbourke/tensorflow-deep-learning/main/extras/helper_functions.py [following]
--2024-01-29 14:01:20--  https://raw.githubusercontent.com/mrdbourke/tensorflow-deep-learning/main/extras/helper_functions.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 10246 (10K) [text/plain]
Saving to: ‘helper_functions.py’

helper_functions.py 100%[===================>]  10.01K  --.-KB/s    in 0s      

2024-01-29 14:01:20 (67.6 MB/s) - ‘helper_func

## Get a text dataset

- Using Kaggle's Introduction to NLP dataset

In [3]:
!wget https://storage.googleapis.com/ztm_tf_course/nlp_getting_started.zip
# Uzip data
unzip_data("nlp_getting_started.zip")

--2024-01-29 14:01:29--  https://storage.googleapis.com/ztm_tf_course/nlp_getting_started.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 64.233.182.207, 64.233.183.207, 173.194.193.207, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|64.233.182.207|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 607343 (593K) [application/zip]
Saving to: ‘nlp_getting_started.zip’

nlp_getting_started 100%[===================>] 593.11K  --.-KB/s    in 0.008s  

2024-01-29 14:01:29 (73.0 MB/s) - ‘nlp_getting_started.zip’ saved [607343/607343]



## Visualizing a text dataset

In [4]:
import pandas as pd
train_df = pd.read_csv("train.csv")
test_df = pd.read_csv("test.csv")
train_df.head()

,id,keyword,location,text,target
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1


In [5]:
train_df["text"][100]

'.@NorwayMFA #Bahrain police had previously died in a road accident they were not killed by explosion https://t.co/gFJfgTodad'

In [6]:
# Shuffle training datafram
train_df_shuffled = train_df.sample(frac=1)
train_df_shuffled.head()

,id,keyword,location,text,target
3308,4739,evacuate,NaN,The U.S. also flew over each bomb site in Worl...,1
4433,6306,hostage,????,whO'S THAT SHADOW HOLDIN ME HOSTAGE I'VE BEEN ...,0
2491,3578,desolate,NaN,Unexercised honda run-down neighborhood desola...,1
6984,10016,twister,Detroit,Crazy Mom Threw Teen Daughter a NUDE Twister S...,0
4192,5955,hazard,Chicago,CONFIRMED: Sanchez Hazard and Bolasie will be ...,1


In [7]:
# What does the test dataframe look like?
test_df.head()

,id,keyword,location,text
0,0,NaN,NaN,Just happened a terrible car crash
1,2,NaN,NaN,"Heard about #earthquake is different cities, s..."
2,3,NaN,NaN,"there is a forest fire at spot pond, geese are..."
3,9,NaN,NaN,Apocalypse lighting. #Spokane #wildfires
4,11,NaN,NaN,Typhoon Soudelor kills 28 in China and Taiwan


In [8]:
# How many examples of each class are there?
train_df.target.value_counts()

0    4342
1    3271
Name: target, dtype: int64

In [9]:
# How many locations?
train_df.location.value_counts()

USA                    104
New York                71
United States           50
London                  45
Canada                  29
                      ... 
MontrÌ©al, QuÌ©bec       1
Montreal                 1
ÌÏT: 6.4682,3.18287      1
Live4Heed??              1
Lincoln                  1
Name: location, Length: 3341, dtype: int64

In [10]:
# How many total samples?
len(train_df), len(test_df)

(7613, 3263)

In [11]:
# Let's visualize some random training examples
import random
random_index = random.randint(0, len(train_df)-5)
for row in train_df_shuffled[["text", "target"]][random_index:random_index+5].itertuples():
  _, text, target = row
  print(f"Target: {target}", r"(real disaster)" if target > 0 else r"(not real disaster)")
  print(f"Text:\n{text}\n")
  print("---\n")

Target: 0 (not real disaster)
Text:
@Surf_Photo I squeezed a tear. Famine plague tsunami no chance. Lego - blubber!

---

Target: 1 (real disaster)
Text:
Emergency crews respond to chemical spill downtown beaumont #benews http://t.co/PME0HOJVYA

---

Target: 0 (not real disaster)
Text:
dogs Lightning reshapes rocks at the atomic level - A lightning strike can reshape a mineral's crystal structure ... http://t.co/2Wvmij5SA4

---

Target: 1 (real disaster)
Text:
RT @TrueDiagnosis: 250K deaths per year from #physician error: http://t.co/DUtYzQR2P7åÊ How to avoid being one of them: http://t.co/OznsxxvxÛ_

---

Target: 0 (not real disaster)
Text:
Are Users of this Sub to be Quarantined? http://t.co/9nLY2TovUD

---



In [12]:
print("Hello\'\\ \n")

Hello'\ 



In [13]:
print(r"Hello\'\\ \n")

Hello\'\\ \n


### Split data into training and validation sets

In [14]:
from sklearn.model_selection import train_test_split
train_sentences, validation_sentences, train_labels, val_labels = train_test_split(train_df_shuffled["text"].to_numpy(),
                                                                                   train_df_shuffled["target"].to_numpy(),
                                                                                   test_size=0.1)


In [15]:
# Check the lengths
len(train_sentences), len(train_labels), len(validation_sentences), len(val_labels)

(6851, 6851, 762, 762)

In [16]:
# Check the first 10 samples
train_sentences[:10], train_labels[:10]

(array(['Walmart is taking steps to keep children safe in hot vehicles. Take a look at the innovative car seat here! http://t.co/z3nEvGlUFm',
        'smokers that ruin that new car smell ????',
        'DHS Refuses to Call Chattanooga \x89Û÷Islamic Terrorism\x89Ûª out of respect for MUSLIMS ... http://t.co/u8RGB51d22 via @po_st http://t.co/2tnu95VGFE',
        "If you ever think you running out of choices in life rembr there's that kid that has no choice but wear a suicide bomb vest",
        '@DrMartyFox     In the U.S. government and Libs made evil good and good evil. We will reap the whirlwind. Lord have mercy on us.',
        '#Sinjar Massacre #Yazidis Blast Lack of Action Over Hostages http://t.co/JhOaHpbpQ4 Portland #Phoenix #Newyork #Miami #Atlanta #Casper #Iraq',
        'yhngsjlg just tweeted about their secret crush:http://t.co/IoqM5bm1Dg',
        'WWI WWII JAPANESE ARMY NAVY MILITARY JAPAN LEATHER WATCH WAR MIDO WW1 2 - Full read by eBay http://t.co/QUmcE7W2tY http://t.co/

## Converting text into numbers

In [17]:
# Embedding - every word gets turned into a vector; you can specify the dimensions of the vector
  # gets updated as the network learns; therfore it can learn relationships between tokens
  # Pretrained embedding libraries e.g. word2vec, GloVe, etc.


In [19]:
# Text vectorization (tokenization)
train_sentences[:5]

array(['Walmart is taking steps to keep children safe in hot vehicles. Take a look at the innovative car seat here! http://t.co/z3nEvGlUFm',
       'smokers that ruin that new car smell ????',
       'DHS Refuses to Call Chattanooga \x89Û÷Islamic Terrorism\x89Ûª out of respect for MUSLIMS ... http://t.co/u8RGB51d22 via @po_st http://t.co/2tnu95VGFE',
       "If you ever think you running out of choices in life rembr there's that kid that has no choice but wear a suicide bomb vest",
       '@DrMartyFox     In the U.S. government and Libs made evil good and good evil. We will reap the whirlwind. Lord have mercy on us.'],
      dtype=object)

In [22]:
import tensorflow as tf
from tensorflow.keras.layers.experimental.preprocessing import TextVectorization
# use the default textvectorization parameters
text_vectorizer = TextVectorization(max_tokens=None, # how many words in our vocabulary; no cap on size of vocab
                                    standardize='lower_and_strip_punctuation',
                                    split='whitespace',
                                    ngrams=None, # create groups of n-words?
                                    output_mode='int', # how to map tokens to numbers
                                    output_sequence_length=None,
                                    pad_to_max_tokens=False)

In [25]:
# find the average number of tokens in the training tweets
round((sum([len(i.split()) for i in train_sentences]))/len(train_sentences))

15

In [26]:
# Setup text vectorization variables
max_vocab_length = 10000 # max num of words to have in vocab
max_length = 15 # max length our sequences will be

text_vectorizer = TextVectorization(max_tokens = max_vocab_length,
                                    output_mode='int',
                                    output_sequence_length=max_length)

In [27]:
# Fit the text vectorizer to the training text
text_vectorizer.adapt(train_sentences)

In [28]:
# Create a sample sentence and tokenize it
sample_sentence = "There's a flood in my street!"
text_vectorizer([sample_sentence])

<tf.Tensor: shape=(1, 15), dtype=int64, numpy=
array([[285,   3, 203,   5,  13, 760,   0,   0,   0,   0,   0,   0,   0,
          0,   0]])>

In [29]:
# Choose a random sentence from the training dataset and tokenize it
random_sentence = random.choice(train_sentences)
print(f"Original text:\n{random_sentence} \
      \n\nVectorized versions:")
text_vectorizer([random_sentence])

Original text:
An Eau Claire man who police said was drunk when his SUV collided with a train was sentenced in Chippewa County: http://t.co/kQpkY7Dthj       

Vectorized versions:


<tf.Tensor: shape=(1, 15), dtype=int64, numpy=
array([[  47,    1,    1,   96,   65,   76,  207,   23, 1409,   42,   82,
        3359,  289,   14,    3]])>

In [31]:
# Get the unique words in the vocab
words_in_vocab = text_vectorizer.get_vocabulary() # Get all the unique words in our training data
# Most common words
top_5_words = words_in_vocab[:5]
bottom_5_words = words_in_vocab[-5:]
print(f"Number of words in vocab: {len(words_in_vocab)}")
print(f"5 most common words: {top_5_words}")
print(f"5 least common words: {bottom_5_words}")

Number of words in vocab: 10000
5 most common words: ['', '[UNK]', 'the', 'a', 'to']
5 least common words: ['pampalmater', 'palmer', 'palm', 'palinfoen', 'palestinian\x89Û']


### Creating an Embedding using an Embedding Layer

In [32]:
# taken tokenized text into embedding vectors
from tensorflow.keras import layers
embedding = layers.Embedding(input_dim=max_vocab_length,
                             output_dim=128, # best to choose number divisible by 8
                             input_length=max_length # how long is each input
                             )
embedding

In [34]:
random_sentence = random.choice(train_sentences)
print(f"Original text:\n {random_sentence}\
\n\n Embedded version:")

sample_embed = embedding(text_vectorizer([random_sentence]))
sample_embed

Original text:
 Angry Mistrustful Rescued Elephant Finds Peace and Friendship in Her New Home (PHOTOS) http://t.co/VaUnPS6WJa via @OneGreenPlanet

 Embedded version:


<tf.Tensor: shape=(1, 15, 128), dtype=float32, numpy=
array([[[ 0.0119215 , -0.01844446,  0.01608418, ...,  0.0331471 ,
          0.04622347, -0.00965104],
        [-0.01976824, -0.00987335, -0.00597851, ...,  0.00615456,
          0.00902549,  0.02964136],
        [ 0.01170846, -0.00346463,  0.04439021, ...,  0.01210959,
         -0.04139175,  0.04038895],
        ...,
        [-0.03757245, -0.04123909, -0.0207515 , ..., -0.04324029,
         -0.04467732, -0.04026509],
        [-0.01976824, -0.00987335, -0.00597851, ...,  0.00615456,
          0.00902549,  0.02964136],
        [ 0.04656685,  0.00707588, -0.02877348, ...,  0.03993856,
          0.00126965,  0.03916726]]], dtype=float32)>

In [35]:
# Check out a single token's embedding
sample_embed[0][0], sample_embed[0][0].shape, random_sentence[0]

(<tf.Tensor: shape=(128,), dtype=float32, numpy=
 array([ 0.0119215 , -0.01844446,  0.01608418, -0.0314784 , -0.00424819,
         0.02723977,  0.04408084,  0.02263815, -0.01819084, -0.0406514 ,
         0.04497233, -0.0199267 ,  0.02046588, -0.01974303,  0.03180161,
         0.03016042, -0.03184827, -0.02237523, -0.03365644,  0.00900578,
        -0.03635205,  0.04647831,  0.00195167,  0.03791559,  0.04936734,
         0.04619019,  0.01352959, -0.01242931, -0.03455772,  0.04259647,
         0.00590353, -0.03785444, -0.00836881,  0.01208258,  0.04478769,
        -0.04913454, -0.00471078, -0.00231642,  0.00862597,  0.02782747,
         0.0312209 ,  0.03733075, -0.00243094, -0.00921518,  0.00109779,
        -0.03203968, -0.01617962,  0.02995736,  0.00590527, -0.04743312,
         0.03510535, -0.01668534, -0.02126577,  0.03157083,  0.00695102,
         0.04971497,  0.02388363, -0.02539698, -0.03241462,  0.00720228,
         0.02679218,  0.04816543, -0.00313191,  0.00417011, -0.00878793,
  

## Modeling a text dataset
* Model 0: Naive Bayes (baseline) [sklearn ML map](https://scikit-learn.org/stable/tutorial/machine_learning_map/index.html)
* Model 2: LSTM model (RNN)
* Model 3: GRU model (RNN)
* Model 4: Bidirectional-LSTM model (RNN)
* Model 5: 1D Convolutional NN (CNN)
* Model 6: TF Hub Pretrained Feature Extractor (transfer learning)
* Model 7: Same as model 6 with 10% of training data

### Model 0: Getting a baseline

In [36]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.feature_extraction.text import TfidfVectorizer # turn text into numbers/vectors
from sklearn.pipeline import Pipeline
model_0 = Pipeline([
    ("tfidf", TfidfVectorizer()),
    ("clf", MultinomialNB()) # clf == "classifier"
])
model_0.fit(train_sentences, train_labels)

Pipeline(steps=[('tfidf', TfidfVectorizer()), ('clf', MultinomialNB())])

In [37]:
# Evaluate our baseline model
baseline_score = model_0.score(validation_sentences, val_labels)
print(f"Our baseline model achieves an accuracy of: {baseline_score*100:.2f}%")

Our baseline model achieves an accuracy of: 82.15%


In [39]:
train_df.target.value_counts()

0    4342
1    3271
Name: target, dtype: int64

In [40]:
# Make predictions
baseline_preds = model_0.predict(validation_sentences)
baseline_preds[:20]

array([0, 0, 0, 1, 1, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 1, 0])

"Humboldt Cty Sheriff's Office has issued an evacuation advisory for 10 residence in the Lassics area... more info at http://t.co/ERUzBUQZYU"